<a href="https://colab.research.google.com/github/melkatewabe10/Machine-learning_LST-Estimation-/blob/main/Data_downloading.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Library

In [6]:
import ee
import folium
import matplotlib.pyplot as plt
import geemap
import math
# Authenticate to Earth Engine. This will open a browser window.
ee.Authenticate()

# Initialize the Earth Engine API.
ee.Initialize(project='ee-mftewabe')

ALB

In [ ]:
# -------------------------------------------------------------------
# 1. Define the study region (Taiwan)
# -------------------------------------------------------------------
gaul = ee.FeatureCollection("FAO/GAUL/2015/level0")
taiwan = gaul.filter(ee.Filter.eq('ADM0_NAME', 'Taiwan'))

# -------------------------------------------------------------------
# 2. Load ESA WorldCover and create a land mask
# -------------------------------------------------------------------
worldcover = ee.ImageCollection("ESA/WorldCover/v200").first().clip(taiwan)
land_mask = worldcover.neq(50).And(worldcover.neq(80))  # Remove built-up (50) and water (80)

# -------------------------------------------------------------------
# 3. MODIS QA Filtering Function
# -------------------------------------------------------------------
# def mask_modis_qa(image):
#     qa = image.select('BRDF_Albedo_Band_Mandatory_Quality_Band1')  # MODIS MYD09A1 QA band
#     modland_qa = qa.bitwiseAnd(1)  # Extract bits 0–1
#     mask = modland_qa.lte(1)  # Keep only good pixels (0 or 1)
#     return image.updateMask(mask)
# -------------------------------------------------------------------
# 4. Define Seasons
# -------------------------------------------------------------------
seasons = {
    "Spring": ('03-01', '05-31'),
    "Summer": ('06-01', '08-31'),
    "Autumn": ('09-01', '11-30'),
    "Winter": ('12-01', '02-28')  # Winter spans two years
}
season_codes = {
    "Spring": "02",
    "Summer": "03",
    "Autumn": "04",
    "Winter": "01"
}
# -------------------------------------------------------------------
# 5. Loop through each year and export seasonal ALB
# -------------------------------------------------------------------
for year in range(2017, 2025):
    print(f"Processing year: {year}")

    for season, (start, end) in seasons.items():
        print(f"  Processing {season} of {year}")

        if season == "Winter":
            start_date = ee.Date(f"{year}-12-01")
            end_date = ee.Date(f"{year + 1}-02-28")
        else:
            start_date = ee.Date(f"{year}-{start}")
            end_date = ee.Date(f"{year}-{end}")

        # Load, filter, and apply QA mask
        alb_collection = (ee.ImageCollection("MODIS/061/MCD43A3")
            .filterDate(start_date, end_date)
            .filterBounds(taiwan)
            # .map(mask_modis_qa)
            .select('Albedo_WSA_Band1')
            .median()
            .clip(taiwan)
            )

        # Convert to scaled reflectance
        ALB_scaled = alb_collection.multiply(0.001)

        # # Apply land cover mask
        ALB_masked = ALB_scaled.updateMask(land_mask)
        # Use numeric code for naming
        season_code = season_codes[season]

        # Export
        task = ee.batch.Export.image.toDrive(
            image=ALB_masked,
            description=f'ALB_{year}_{season_code}',
            folder='SEASON_THREE',
            fileNamePrefix=f'ALB_{year}_{season_code}',
            region=taiwan.geometry().bounds().getInfo()['coordinates'],
            scale=1000,
            crs='EPSG:32651',
            maxPixels=1e13
        )

        task.start()
        print(f"    Export task for {year} - {season} started.")


DSR

In [ ]:
# -------------------------------------------------------------------
# 1. Define the study region (Taiwan)
# -------------------------------------------------------------------
gaul = ee.FeatureCollection("FAO/GAUL/2015/level0")
taiwan = gaul.filter(ee.Filter.eq('ADM0_NAME', 'Taiwan'))

# -------------------------------------------------------------------
# 2. Load ESA WorldCover and create a land mask
# -------------------------------------------------------------------
worldcover = ee.ImageCollection("ESA/WorldCover/v200").first().clip(taiwan)
land_mask = worldcover.neq(50).And(worldcover.neq(80))  # Remove built-up (50) and water (80)

# -------------------------------------------------------------------
# 3. MODIS QA Filtering Function
# -------------------------------------------------------------------
# def mask_modis_qa(image):
#     qa = image.select('DSR_Quality')  # MODIS MYD09A1 QA band
#     modland_qa = qa.bitwiseAnd(1)  # Extract bits 0–1
#     mask = modland_qa.lte(1)  # Keep only good pixels (0 or 1)
#     return image.updateMask(mask)
# -------------------------------------------------------------------
# 4. Define Seasons
# -------------------------------------------------------------------
seasons = {
    "Spring": ('03-01', '05-31'),
    "Summer": ('06-01', '08-31'),
    "Autumn": ('09-01', '11-30'),
    "Winter": ('12-01', '02-28')  # Winter spans two years
}
season_codes = {
    "Spring": "02",
    "Summer": "03",
    "Autumn": "04",
    "Winter": "01"
}
# -------------------------------------------------------------------
# 5. Loop through each year and export seasonal DSR
# -------------------------------------------------------------------
for year in range(2017, 2025):
    print(f"Processing year: {year}")

    for season, (start, end) in seasons.items():
        print(f"  Processing {season} of {year}")

        if season == "Winter":
            start_date = ee.Date(f"{year}-12-01")
            end_date = ee.Date(f"{year + 1}-02-28")
        else:
            start_date = ee.Date(f"{year}-{start}")
            end_date = ee.Date(f"{year}-{end}")

        # Load, filter, and apply QA mask
        DSR_collection = (ee.ImageCollection("MODIS/062/MCD18A1")
            .filterDate(start_date, end_date)
            .filterBounds(taiwan)
            # .map(mask_modis_qa)
            .select('GMT_0600_DSR')
            .mean()
            .clip(taiwan)
            )

        # # Convert to scaled reflectance
        # DSR_scaled = DSR_collection.multiply(0.001)

        # Apply land cover mask
        DSR_masked = DSR_collection.updateMask(land_mask)
        # # Use numeric code for naming
        season_code = season_codes[season]

        # Export
        task = ee.batch.Export.image.toDrive(
            image=DSR_masked,
            description=f'DSR_{year}_{season_code}',
            folder='SEASON_THREE',
            fileNamePrefix=f'DSR_{year}_{season_code}',
            region=taiwan.geometry().bounds().getInfo()['coordinates'],
            scale=1000,
            crs='EPSG:32651',
            maxPixels=1e13
        )

        task.start()
        print(f"    Export task for {year} - {season} started.")


LAI

In [ ]:
# -------------------------------------------------------------------
# 1. Define the study region (Taiwan)
# -------------------------------------------------------------------
gaul = ee.FeatureCollection("FAO/GAUL/2015/level0")
taiwan = gaul.filter(ee.Filter.eq('ADM0_NAME', 'Taiwan'))

# -------------------------------------------------------------------
# 2. Load ESA WorldCover and create a land mask
# -------------------------------------------------------------------
worldcover = ee.ImageCollection("ESA/WorldCover/v200").first().clip(taiwan)
land_mask = worldcover.neq(50).And(worldcover.neq(80))  # Remove built-up (50) and water (80)

# -------------------------------------------------------------------
# 3. MODIS QA Filtering Function
# -------------------------------------------------------------------
# def mask_modis_qa(image):
#     qa = image.select('FparLai_QC')  # MODIS MYD09A1 QA band
#     modland_qa = qa.bitwiseAnd(25)  # Extract bits 0–1
#     mask = modland_qa.lte(0)  # Keep only good pixels (0 or 1)
#     return image.updateMask(mask)

# def mask_modis_qa(image):
#     qa = image.select('FparLai_QC')

#     # Extract bits 0–1 (MODLAND QA bits)
#     modland_qa = qa.bitwiseAnd(3)  # 0b000000000011
#     modland_mask = modland_qa.lte(0)  # Keep if 0 (ideal) or 1 (good)

#     # # Extract bit 2 (data quality flag)
#     # data_quality = qa.bitwiseAnd(4).rightShift(2)  # 0b000000000100
#     # data_quality_mask = data_quality.lte(0)  # Keep if 0 (best quality)

#     # Extract bit 10 (adjacent cloud detected)
#     cloud_state = qa.bitwiseAnd(24).rightShift(10)  # 0b0000010000000000
#     cloud_state_mask = cloud_state.lte(0)  # Keep if no state cloud

#     # Combine all masks
#     combined_mask = modland_mask.And(cloud_state_mask)
#     # And(data_quality_mask)

#     # Apply the mask
#     return image.updateMask(combined_mask)
# -------------------------------------------------------------------
# 4. Define Seasons
# -------------------------------------------------------------------
seasons = {
    "Spring": ('03-01', '05-31'),
    "Summer": ('06-01', '08-31'),
    "Autumn": ('09-01', '11-30'),
    "Winter": ('12-01', '02-28')  # Winter spans two years
}
season_codes = {
    "Spring": "02",
    "Summer": "03",
    "Autumn": "04",
    "Winter": "01"
}
# -------------------------------------------------------------------
# 5. Loop through each year and export seasonal LAI
# -------------------------------------------------------------------
for year in range(2017, 2025):
    print(f"Processing year: {year}")

    for season, (start, end) in seasons.items():
        print(f"  Processing {season} of {year}")

        if season == "Winter":
            start_date = ee.Date(f"{year}-12-01")
            end_date = ee.Date(f"{year + 1}-02-28")
        else:
            start_date = ee.Date(f"{year}-{start}")
            end_date = ee.Date(f"{year}-{end}")

        # Load, filter, and apply QA mask
        LAI_collection = (ee.ImageCollection("MODIS/061/MOD15A2H")
            .filterDate(start_date, end_date)
            .filterBounds(taiwan)
            # .map(mask_modis_qa)
            .select('Lai_500m')
            .median()
            .clip(taiwan)
            )
  #  MODIS/061/MOD15A2H
        # Convert to scaled reflectance
        LAI_scaled = LAI_collection.multiply(0.1).clamp(0, 7.5)

        # # Apply land cover mask
        # LAI_masked = LAI_scaled.updateMask(land_mask)
        # # Use numeric code for naming
        season_code = season_codes[season]

        # Export
        task = ee.batch.Export.image.toDrive(
            image=LAI_scaled,
            description=f'LAI_{year}_{season_code}',
            folder='SEASON_THREE',
            fileNamePrefix=f'LAI_{year}_{season_code}',
            region=taiwan.geometry().bounds().getInfo()['coordinates'],
            scale=1000,
            crs='EPSG:32651',
            maxPixels=1e13
        )

        task.start()
        print(f"    Export task for {year} - {season} started.")


LST

In [ ]:
# -------------------------------------------------------------------
# 1. Define the study region (Taiwan)
# -------------------------------------------------------------------
gaul = ee.FeatureCollection("FAO/GAUL/2015/level0")
taiwan = gaul.filter(ee.Filter.eq('ADM0_NAME', 'Taiwan'))

# -------------------------------------------------------------------
# 2. Load ESA WorldCover and create a land mask
# -------------------------------------------------------------------
worldcover = ee.ImageCollection("ESA/WorldCover/v200").first().clip(taiwan)
land_mask = worldcover.neq(50).And(worldcover.neq(80))  # Remove built-up (50) and water (80)

# # -------------------------------------------------------------------
# # 3. MODIS QA Filtering Function
# # -------------------------------------------------------------------
# def mask_modis_qa(image):
#     qa = image.select('QC_Day')  # MODIS MYD09A1 QA band
#     modland_qa = qa.bitwiseAnd(3)  # Extract bits 0–1
#     mask = modland_qa.lte(0)  # Keep only good pixels (0 or 1)
#     return image.updateMask(mask)
# -------------------------------------------------------------------
# 4. Define Seasons
# -------------------------------------------------------------------
seasons = {
    "Spring": ('03-01', '05-31'),
    "Summer": ('06-01', '08-31'),
    "Autumn": ('09-01', '11-30'),
    "Winter": ('12-01', '02-28')  # Winter spans two years
}
season_codes = {
    "Spring": "02",
    "Summer": "03",
    "Autumn": "04",
    "Winter": "01"
}
# -------------------------------------------------------------------
# 5. Loop through each year and export seasonal ALB
# -------------------------------------------------------------------
for year in range(2017, 2025):
    print(f"Processing year: {year}")

    for season, (start, end) in seasons.items():
        print(f"  Processing {season} of {year}")

        if season == "Winter":
            start_date = ee.Date(f"{year}-12-01")
            end_date = ee.Date(f"{year + 1}-02-28")
        else:
            start_date = ee.Date(f"{year}-{start}")
            end_date = ee.Date(f"{year}-{end}")

        # Load, filter, and apply QA mask
        LST_collection = (ee.ImageCollection("MODIS/061/MOD11A1")
            .filterDate(start_date, end_date)
            .filterBounds(taiwan)
            # .map(mask_modis_qa)
            .select('LST_Day_1km')
            .median()
            .clip(taiwan)
            )

        # Convert to scaled reflectance
        LST_scaled = LST_collection.multiply(0.02)

        # # Apply land cover mask
        LST_masked = LST_scaled.updateMask(land_mask)
        # Use numeric code for naming
        season_code = season_codes[season]


        # Export
        task = ee.batch.Export.image.toDrive(
            image=LST_masked,
            description=f'LST_{year}_{season_code}',
            folder='SEASON_THREE',
            fileNamePrefix=f'LST_{year}_{season_code}',
            region=taiwan.geometry().bounds().getInfo()['coordinates'],
            scale=1000,
            crs='EPSG:32651',
            maxPixels=1e13
        )

        task.start()
        print(f"    Export task for {year} - {season} started.")


NDWI

In [ ]:

# -------------------------------------------------------------------
# 1. Define the study region (Taiwan)
# -------------------------------------------------------------------
gaul = ee.FeatureCollection("FAO/GAUL/2015/level0")
taiwan = gaul.filter(ee.Filter.eq('ADM0_NAME', 'Taiwan'))

# -------------------------------------------------------------------
# 2. Load ESA WorldCover and create a mask
# # -------------------------------------------------------------------
worldcover = ee.ImageCollection("ESA/WorldCover/v200").first().clip(taiwan)
land_mask = worldcover.neq(50).And(worldcover.neq(80))  # Remove built-up (50) and water (80)

# -------------------------------------------------------------------
# 3. Define cloud masking and NDWI computation functions.
# -------------------------------------------------------------------
def mask_modis_qa(image):
    qa = image.select('StateQA')  # MODIS MYD09A1 QA band
    modland_qa = qa.bitwiseAnd(3)  # Extract bits 0–1
    mask = modland_qa.lte(0)  # Keep only good pixels (0 or 1)
    return image.updateMask(mask)


# def mask_modis_qa(image):
#     qa = image.select('StateQA')

#     # Extract bits 0–1 (MODLAND QA bits)
#     modland_qa = qa.bitwiseAnd(3)  # 0b000000000011
#     modland_mask = modland_qa.lte(0)  # Keep if 0 (ideal) or 1 (good)

    # # Extract bit 2 (data quality flag)
    # data_quality = qa.bitwiseAnd(4).rightShift(2)  # 0b000000000100
    # data_quality_mask = data_quality.lte(0)  # Keep if 0 (best quality)

    # # Extract bit 10 (adjacent cloud detected)
    # cloud_adjacent = qa.bitwiseAnd(1024).rightShift(10)  # 0b0000010000000000
    # cloud_adjacent_mask = cloud_adjacent.lte(0)  # Keep if no adjacent cloud

    # # Combine all masks
    # combined_mask = modland_mask.And(cloud_adjacent_mask)
    # And(data_quality_mask)

    # # Apply the mask
    # return image.updateMask(combined_mask)


def compute_ndwi(image):
    nir = image.select('sur_refl_b02').multiply(0.0001)  # Convert scaled reflectance
    swir = image.select('sur_refl_b07').multiply(0.0001)
    ndwi = nir.subtract(swir).divide(nir.add(swir)).rename('NDWI')
    return image.addBands(ndwi)

# -------------------------------------------------------------------
# 4. Define Seasons
# -------------------------------------------------------------------
seasons = {
    "Spring": ('03-01', '05-31'),
    "Summer": ('06-01', '08-31'),
    "Autumn": ('09-01', '11-30'),
    "Winter": ('12-01', '02-28')  # Winter spans two years
}
season_codes = {
    "Spring": "02",
    "Summer": "03",
    "Autumn": "04",
    "Winter": "01"
}
# -------------------------------------------------------------------
# 5. Loop through each year and export seasonal NDWI
# -------------------------------------------------------------------
for year in range(2017, 2025):
    print(f"Processing year: {year}")

    for season, (start, end) in seasons.items():
        print(f"  Processing {season} of {year}")

        if season == "Winter":
            start_date = ee.Date(f"{year}-12-01")
            end_date = ee.Date(f"{year + 1}-02-28")  # Winter spans two years
        else:
            start_date = ee.Date(f"{year}-{start}")
            end_date = ee.Date(f"{year}-{end}")

        # Load MODIS Surface Reflectance data
        ndwi_image = (ee.ImageCollection('MODIS/061/MOD09A1')
                      .filterDate(start_date, end_date)
                      .filterBounds(taiwan)
                      .map(mask_modis_qa)
                      .map(compute_ndwi)
                      .select('NDWI')
                      .median()
                      .clip(taiwan)
                         )

        # # Compute seasonal mean NDWI
        # ndwi_mean = NDWI_collection.select('NDWI').mean()

        # )
        # # Compute seasonal mean NDWI
        # collection= collection.map(mask_modis_qa).map(compute_ndwi)

        # Compute seasonal mean NDWI
        ndwi_masked =ndwi_image.updateMask(land_mask).clamp(-1, 1)


        # # Apply land mask
        season_code = season_codes[season]

        # Export to Google Drive
        task = ee.batch.Export.image.toDrive(
            image= ndwi_masked,
            description=f'NDWI_{year}_{season_code}',
            folder='SEASON_THREE',
            fileNamePrefix=f'NDWI_{year}_{season_code}',
            region=taiwan.geometry().bounds().getInfo()['coordinates'],
            scale=1000,  # MODIS resolution (~1 km)
            crs='EPSG:32651',
            maxPixels=1e13
        )

        task.start()
        print(f"Export task for {year} - {season} started. ")

# NDVI

In [ ]:
# -------------------------------------------------------------------
# 1. Define the study region (Taiwan)
# -------------------------------------------------------------------
gaul = ee.FeatureCollection("FAO/GAUL/2015/level0")
taiwan = gaul.filter(ee.Filter.eq('ADM0_NAME', 'Taiwan'))

# -------------------------------------------------------------------
# 2. Load ESA WorldCover and create a mask (exclude water and built-up)
# -------------------------------------------------------------------
worldcover = ee.ImageCollection("ESA/WorldCover/v200").first().clip(taiwan)
land_mask = worldcover.neq(80).And(worldcover.neq(80))  # Remove built-up (50) and water (80)

# -------------------------------------------------------------------
# 3. Define cloud masking and NDVI computation functions
# -------------------------------------------------------------------
def mask_modis_qa(image):
    qa = image.select('StateQA')  # MODIS MYD09A1 QA band
    modland_qa = qa.bitwiseAnd(3)  # Extract bits 0–1
    mask = modland_qa.lte(0)  # Keep only good pixels (0 or 1)
    return image.updateMask(mask)

def compute_ndvi(image):
    red = image.select('sur_refl_b01').multiply(0.0001)
    nir = image.select('sur_refl_b02').multiply(0.0001)
    ndvi = nir.subtract(red).divide(nir.add(red)).rename('NDVI')
    return image.addBands(ndvi)

# -------------------------------------------------------------------
# 4. Define seasons
# -------------------------------------------------------------------
seasons = {
    "Spring": ('03-01', '05-31'),
    "Summer": ('06-01', '08-31'),
    "Autumn": ('09-01', '11-30'),
    "Winter": ('12-01', '02-28')  # Winter spans two years
}
season_codes = {
    "Spring": "02",
    "Summer": "03",
    "Autumn": "04",
    "Winter": "01"
}

# -------------------------------------------------------------------
# 5. Loop through years and export seasonal NDVI
# -------------------------------------------------------------------
for year in range(2000, 2009):
    print(f"Processing year: {year}")

    for season, (start, end) in seasons.items():
        print(f"  Processing {season} of {year}")

        if season == "Winter":
            start_date = ee.Date(f"{year}-12-01")
            end_date = ee.Date(f"{year + 1}-02-28")
        else:
            start_date = ee.Date(f"{year}-{start}")
            end_date = ee.Date(f"{year}-{end}")

        # Load MODIS Surface Reflectance data
        ndvi_image = (ee.ImageCollection('MODIS/061/MOD09A1')
                           .filterDate(start_date, end_date)
                           .filterBounds(taiwan)
                           .map(mask_modis_qa)
                           .map(compute_ndvi)
                           .select('NDVI')
                           .median()
                           .clip(taiwan)
                                )

        # Mask out water and built-up areas
        ndvi_masked = ndvi_image.updateMask(land_mask).clamp(-1, 1)

        # Define season code
        season_code = season_codes[season]

        # Export to Google Drive
        task = ee.batch.Export.image.toDrive(
            image=ndvi_masked,
            description=f'NDVI_{year}_{season_code}',
            folder='SEASON_ONE_SWAT',
            fileNamePrefix=f'NDVI_{year}_{season_code}',
            region=taiwan.geometry().bounds().getInfo()['coordinates'],
            scale=1000,
            crs='EPSG:32651',
            maxPixels=1e13
        )

        task.start()
        print(f"Export task for {year} - {season} started.")


#EVI

In [ ]:

# -------------------------------------------------------------------
# 1. Define the study region (Taiwan)
# -------------------------------------------------------------------
gaul = ee.FeatureCollection("FAO/GAUL/2015/level0")
taiwan = gaul.filter(ee.Filter.eq('ADM0_NAME', 'Taiwan'))

# -------------------------------------------------------------------
# 2. Load ESA WorldCover and create a mask
# -------------------------------------------------------------------
worldcover = ee.ImageCollection("ESA/WorldCover/v200").first().clip(taiwan)
land_mask = worldcover.neq(50).And(worldcover.neq(80))  # Remove built-up (50) and water (80)

# -------------------------------------------------------------------
# 3. Define cloud masking and EVI computation functions.
# -------------------------------------------------------------------
def mask_modis_qa(image):
    qa = image.select('StateQA')  # MODIS MYD09A1 QA band
    modland_qa = qa.bitwiseAnd(3)  # Extract bits 0–1
    mask = modland_qa.lte(0)  # Keep only good pixels (0 or 1)
    return image.updateMask(mask)

# def mask_modis_qa(image):
#     qa = image.select('StateQA')

#     # Extract bits 0–1 (MODLAND QA bits)
#     modland_qa = qa.bitwiseAnd(3)  # 0b000000000011
#     modland_mask = modland_qa.lte(0)  # Keep if 0 (ideal) or 1 (good)

#     # Extract bit 2 (data quality flag)
#     data_quality_b1 = qa.bitwiseAnd(1024).rightShift(10)  # 0b000000000100
#     band_mask = data_quality_b1.lte(0)  # Keep if 0 (best quality)

#     # # Extract bit 10 (adjacent cloud detected)
#     # data_quality_b2 = qa.bitwiseAnd(8192).rightShift(13)  # 0b0000010000000000
#     # band2_mask = data_quality_b2.lte(0)  # Keep if no adjacent cloud

#     # # Extract bit 10 (adjacent cloud detected)
#     # data_quality_b3 = qa.bitwiseAnd(4).rightShift(2)  # 0b0000010000000000
#     # band3_mask = data_quality_b3.lte(0)  # Keep if no adjacent cloud

#     # Combine all masks
#     combined_mask = modland_mask.And(band_mask)
#     # And(data_quality_mask)

#     # Apply the mask
#     return image.updateMask(combined_mask)

def compute_evi(image):
    red = image.select('sur_refl_b01').multiply(0.0001)  # Red
    nir = image.select('sur_refl_b02').multiply(0.0001)  # NIR
    blue = image.select('sur_refl_b03').multiply(0.0001)  # Blue

    G = 2.5
    L = 1.0
    C1 = 6.0
    C2 = 7.5

    numerator = nir.subtract(red)
    denominator = nir.add(red.multiply(C1)).subtract(blue.multiply(C2)).add(L)

    evi = numerator.divide(denominator).multiply(G).rename('EVI')
    return image.addBands(evi)

# -------------------------------------------------------------------
# 4. Define Seasons
# -------------------------------------------------------------------
seasons = {
    "Spring": ('03-01', '05-31'),
    "Summer": ('06-01', '08-31'),
    "Autumn": ('09-01', '11-30'),
    "Winter": ('12-01', '02-28')  # Winter spans two years
}
season_codes = {
    "Spring": "02",
    "Summer": "03",
    "Autumn": "04",
    "Winter": "01"
}
# -------------------------------------------------------------------
# 5. Loop through each year and export seasonal EVI
# -------------------------------------------------------------------
for year in range(2017, 2025):
    print(f"Processing year: {year}")

    for season, (start, end) in seasons.items():
        print(f"  Processing {season} of {year}")

        if season == "Winter":
            start_date = ee.Date(f"{year}-12-01")
            end_date = ee.Date(f"{year + 1}-02-28")  # Winter spans two years
        else:
            start_date = ee.Date(f"{year}-{start}")
            end_date = ee.Date(f"{year}-{end}")

        # Load MODIS Surface Reflectance data
        evi_image = (ee.ImageCollection('MODIS/061/MOD09A1')
                      .filterDate(start_date, end_date)
                      .filterBounds(taiwan)
                      .map(mask_modis_qa)
                      # .map(mask_modis_mqa)
                      .map(compute_evi)
                      .select('EVI')
                      .median()
                      .clip(taiwan)
                                          )

        # # Compute seasonal mean EVI
        # EVI_mean = EVI_collection.select('EVI').mean()

        # )
        # # Compute seasonal mean EVI
        # collection= collection.map(mask_modis_qa).map(compute_EVI)

        # # Compute seasonal mean EVI
        evi_masked = evi_image.updateMask(land_mask).clamp(-1, 1)


        # # Apply land mask
        season_code = season_codes[season]

        # Export to Google Drive
        task = ee.batch.Export.image.toDrive(
            image=evi_masked,
            description=f'EVI_{year}_{season_code}',
            folder='SEASON_THREE',
            fileNamePrefix=f'EVI_{year}_{season_code}',
            region=taiwan.geometry().bounds().getInfo()['coordinates'],
            scale=1000,  # MODIS resolution (~1 km)
            crs='EPSG:32651',
            maxPixels=1e13
        )

        task.start()
        print(f"Export task for {year} - {season} started. ")

# NDLI

In [ ]:
# -------------------------------------------------------------------
# 1. Define the study region (Taiwan)
# -------------------------------------------------------------------
gaul = ee.FeatureCollection("FAO/GAUL/2015/level0")
taiwan = gaul.filter(ee.Filter.eq('ADM0_NAME', 'Taiwan'))

# -------------------------------------------------------------------
# 2. Load ESA WorldCover and create a mask (exclude water and built-up)
# -------------------------------------------------------------------
worldcover = ee.ImageCollection("ESA/WorldCover/v200").first().clip(taiwan)
land_mask = worldcover.neq(50).And(worldcover.neq(80))  # Exclude built-up (50) and water (80)

# -------------------------------------------------------------------
# 3. Define cloud masking and NDLI computation functions
# -------------------------------------------------------------------
def mask_modis_qa(image):
    qa = image.select('StateQA')  # MODIS MYD09A1 QA band
    modland_qa = qa.bitwiseAnd(3)  # Extract bits 0–1
    mask = modland_qa.lte(0)  # Keep only good pixels (0 or 1)
    return image.updateMask(mask)

def compute_ndli(image):
    green = image.select('sur_refl_b04').multiply(0.0001)
    red = image.select('sur_refl_b01').multiply(0.0001)
    swir = image.select('sur_refl_b06').multiply(0.0001)
    ndli = green.subtract(red).divide(green.add(red).add(swir)).rename('NDLI')
    return image.addBands(ndli)

# -------------------------------------------------------------------
# 4. Define seasons
# -------------------------------------------------------------------
seasons = {
    "Spring": ('03-01', '05-31'),
    "Summer": ('06-01', '08-31'),
    "Autumn": ('09-01', '11-30'),
    "Winter": ('12-01', '02-28')  # Winter spans two years
}
season_codes = {
    "Spring": "02",
    "Summer": "03",
    "Autumn": "04",
    "Winter": "01"
}

# -------------------------------------------------------------------
# 5. Loop through years and export seasonal NDLI
# -------------------------------------------------------------------
for year in range(2017, 2025):
    print(f"Processing year: {year}")

    for season, (start, end) in seasons.items():
        print(f"  Processing {season} of {year}")

        if season == "Winter":
            start_date = ee.Date(f"{year}-12-01")
            end_date = ee.Date(f"{year + 1}-02-28")
        else:
            start_date = ee.Date(f"{year}-{start}")
            end_date = ee.Date(f"{year}-{end}")

        # Load MODIS Surface Reflectance data
        ndli_image = (ee.ImageCollection('MODIS/061/MOD09A1')
                           .filterDate(start_date, end_date)
                           .filterBounds(taiwan)
                           .map(mask_modis_qa)
                           .map(compute_ndli)
                           .select('NDLI')
                           .median()
                           .clip(taiwan)
                    )

        # Mask out water and built-up areas
        ndli_masked = ndli_image.updateMask(land_mask).clamp(-1, 1)

        # Define season code
        season_code = season_codes[season]

        # Export to Google Drive
        task = ee.batch.Export.image.toDrive(
            image=ndli_masked,
            description=f'NDLI_{year}_{season_code}',
            folder='SEASON_THREE_SWAT',
            fileNamePrefix=f'NDLI_{year}_{season_code}',
            region=taiwan.geometry().bounds().getInfo()['coordinates'],
            scale=1000,  # MOD09A1 native resolution
            crs='EPSG:32651',
            maxPixels=1e13
        )

        task.start()
        print(f"Export task for {year} - {season} started.")


# ET

In [ ]:
# -------------------------------------------------------------------
# 1. Define the study region (Taiwan)
# -------------------------------------------------------------------
gaul = ee.FeatureCollection("FAO/GAUL/2015/level0")
taiwan = gaul.filter(ee.Filter.eq('ADM0_NAME', 'Taiwan'))

# -------------------------------------------------------------------
# -------------------------------------------------------------------
# 2. Load ESA WorldCover and create a mask (exclude water and built-up)
# -------------------------------------------------------------------
worldcover = ee.ImageCollection("ESA/WorldCover/v200").first().clip(taiwan)
land_mask = worldcover.neq(50).And(worldcover.neq(80))  # Exclude built-up (50) and water (80)

# 3. MODIS ET QA Filtering Function
# -------------------------------------------------------------------
def mask_modis_et_qa(image):
    qa = image.select('ET_QC')
    modland_qa = qa.bitwiseAnd(1)  # Extract bit 0
    mask = modland_qa.lte(0)  # Keep only good quality pixels
    return image.updateMask(mask)

# -------------------------------------------------------------------
# 3. Define Seasons
# -------------------------------------------------------------------
seasons = {
    "Spring": ('03-01', '05-31'),
    "Summer": ('06-01', '08-31'),
    "Autumn": ('09-01', '11-30'),
    "Winter": ('12-01', '02-28')  # Winter spans two years
}
season_codes = {
    "Spring": "02",
    "Summer": "03",
    "Autumn": "04",
    "Winter": "01"
}

# -------------------------------------------------------------------
# 4. Loop through each year and export seasonal ET
# -------------------------------------------------------------------
for year in range(2017, 2025):
    print(f"Processing year: {year}")

    for season, (start, end) in seasons.items():
        print(f"  Processing {season} of {year}")

        if season == "Winter":
            start_date = ee.Date(f"{year}-12-01")
            end_date = ee.Date(f"{year + 1}-02-28")
        else:
            start_date = ee.Date(f"{year}-{start}")
            end_date = ee.Date(f"{year}-{end}")

        # Load, filter, apply QA mask
        ET_image = (ee.ImageCollection("MODIS/061/MOD16A2GF")
            .filterDate(start_date, end_date)
            .filterBounds(taiwan)
            .map(mask_modis_et_qa)
            .select('ET')
            .median()
            .clip(taiwan)
            )

        # Scale ET to mm/8days
        ET_scaled = ET_image.multiply(0.1)
         # Apply land cover mask
        ET_masked = ET_scaled.updateMask(land_mask)

        # Use numeric code for naming
        season_code = season_codes[season]

        # Export
        task = ee.batch.Export.image.toDrive(
            image=ET_masked,
            description=f'ET_{year}_{season_code}',
            folder='VALIDATE_THREE_SWAT',
            fileNamePrefix=f'ET_{year}_{season_code}',
            region=taiwan.geometry().bounds().getInfo()['coordinates'],
            scale=1000,  # MOD16A2 native resolution
            crs='EPSG:32651',
            maxPixels=1e13
        )

        task.start()
        print(f"    Export task for {year} - {season} started.")


# GPP

In [ ]:
# -------------------------------------------------------------------
# 1. Define the study region (Taiwan)
# -------------------------------------------------------------------
gaul = ee.FeatureCollection("FAO/GAUL/2015/level0")
taiwan = gaul.filter(ee.Filter.eq('ADM0_NAME', 'Taiwan'))

# -------------------------------------------------------------------
# 2. Load ESA WorldCover and create a land mask
# -------------------------------------------------------------------
worldcover = ee.ImageCollection("ESA/WorldCover/v200").first().clip(taiwan)
land_mask = worldcover.neq(50).And(worldcover.neq(80))  # Remove built-up (50) and water (80)

# -------------------------------------------------------------------
def mask_modis_gp_qa(image):
    qa = image.select('Psn_QC')
    modland_qa = qa.bitwiseAnd(3)  # Extract bit 0
    mask = modland_qa.lte(0)  # Keep only good quality pixels
    return image.updateMask(mask)

# 3. Define Seasons
# -------------------------------------------------------------------
seasons = {
    "Spring": ('03-01', '05-31'),
    "Summer": ('06-01', '08-31'),
    "Autumn": ('09-01', '11-30'),
    "Winter": ('12-01', '02-28')  # Winter spans two years
}
season_codes = {
    "Spring": "02",
    "Summer": "03",
    "Autumn": "04",
    "Winter": "01"
}

# -------------------------------------------------------------------
# 4. Loop through each year and export seasonal GPP
# -------------------------------------------------------------------
for year in range(2000, 2009):
    print(f"Processing year: {year}")

    for season, (start, end) in seasons.items():
        print(f"  Processing {season} of {year}")

        if season == "Winter":
            start_date = ee.Date(f"{year}-12-01")
            end_date = ee.Date(f"{year + 1}-02-28")
        else:
            start_date = ee.Date(f"{year}-{start}")
            end_date = ee.Date(f"{year}-{end}")

        # Load, filter, and clip
        GPP_collection = (ee.ImageCollection("MODIS/061/MOD17A2H")
            .filterDate(start_date, end_date)
            .filterBounds(taiwan)
            .map(mask_modis_gp_qa)
            .select('Gpp')
            .median()
            .clip(taiwan)
        )

        # Apply scaling and land cover mask
        GPP_scaled = GPP_collection.multiply(0.0001)
        GPP_masked = GPP_scaled.updateMask(land_mask)

        # Use numeric code for naming
        season_code = season_codes[season]

        # Export
        task = ee.batch.Export.image.toDrive(
            image=GPP_masked,
            description=f'GPP_{year}_{season_code}',
            folder='VALIDATE_ONE_SWAT',
            fileNamePrefix=f'GPP_{year}_{season_code}',
            region=taiwan.geometry().bounds().getInfo()['coordinates'],
            scale=1000,
            crs='EPSG:32651',
            maxPixels=1e13
        )

        task.start()
        print(f"    Export task for {year} - {season} started.")


# GPP_yearly

In [ ]:
# -------------------------------------------------------------------
# 1. Define the study region (Taiwan)
# -------------------------------------------------------------------
gaul = ee.FeatureCollection("FAO/GAUL/2015/level0")
taiwan = gaul.filter(ee.Filter.eq('ADM0_NAME', 'Taiwan'))

# -------------------------------------------------------------------
# 2. Load ESA WorldCover and create a land mask
# -------------------------------------------------------------------
worldcover = ee.ImageCollection("ESA/WorldCover/v200").first().clip(taiwan)
land_mask = worldcover.neq(50).And(worldcover.neq(80))  # Remove built-up (50) and water (80)

# -------------------------------------------------------------------
# 3. Loop through each year and export yearly GPP
# -------------------------------------------------------------------
for year in range(2000, 2025):
    print(f"Processing year: {year}")

    start_date = ee.Date(f"{year}-01-01")
    end_date = ee.Date(f"{year}-12-31")

    # Load, filter, and clip
    GPP_collection = (ee.ImageCollection("MODIS/061/MOD17A3HGF")
        .filterDate(start_date, end_date)
        .filterBounds(taiwan)
        .select('Gpp')
        .mean()
        .clip(taiwan)
    )

    # Apply scaling and land cover mask
    GPP_scaled = GPP_collection.multiply(0.0001)
    GPP_masked = GPP_scaled.updateMask(land_mask)

    # Export
    task = ee.batch.Export.image.toDrive(
        image=GPP_masked,
        description=f'GPP_{year}',
        folder='VALIDATE_YEARLY',
        fileNamePrefix=f'GPP_{year}',
        region=taiwan.geometry().bounds().getInfo()['coordinates'],
        scale=1000,
        crs='EPSG:32651',
        maxPixels=1e13
    )

    task.start()
    print(f"  Export task for year {year} started.")



In [ ]:
!pip install rasterio

In [ ]:
import os
import rasterio
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap

# Define a colormap where low values are white and high values are darker
drought_continuous_reversed = LinearSegmentedColormap.from_list(
    'drought_continuous_reversed',
    ['#ffffff', '#ffffff','#ffffff','#ffffff','#fff2b2','#fff2b2','#fff2b2', '#feca79', '#f46d43', '#a50026','#a50026'],  # Light to Dark (low to high)
    N=128  # 256 discrete steps for smooth gradient
)

# # Define reversed colormap: dark (low values) to white (high values)
# drought_continuous = LinearSegmentedColormap.from_list(
#     'drought_continuous',
#     ['#a50026', '#a50026', '#f46d43', '#feca79', '#fff2b2', '#fff2b2', '#fff2b2', '#ffffff', '#ffffff', '#ffffff', '#ffffff'],  # Dark to White
#     N=128
# )

# Path to your LST rasters
folder = "/content/drive/MyDrive/SEASON_I"

for filename in sorted(os.listdir(folder)):
    if filename.endswith(".tif") and "LST" in filename:
        path = os.path.join(folder, filename)

        # Read LST data (temperature data in this case)
        with rasterio.open(path) as src:
            lst = src.read(1).astype(float)
            lst = np.where(lst == src.nodata, np.nan, lst)  # Handle nodata values

        # Display with the reversed colormap (high temperature = dark, low = light)
        plt.figure(figsize=(10, 6))
        img = plt.imshow(lst, cmap=drought_continuous_reversed, vmin=275, vmax=310)  # Adjusted for temperature range

        # Add colorbar with appropriate label
        plt.colorbar(img, label='Land Surface Temperature (LST) [K]')

        # Title, with the axis visible
        plt.title(f"{filename}")
        plt.tight_layout()
        plt.show()


In [ ]:
!pip install rasterio

# Ploting2

In [ ]:
import os
import rasterio
import numpy as np
import matplotlib.pyplot as plt

folder = "/content/drive/MyDrive/VALIDATE_YEARLY"  # Update with your actual NDVI folder path

for filename in sorted(os.listdir(folder)):
    if filename.endswith(".tif") and "GPP" in filename:  # Ensure it's an NDVI file
        path = os.path.join(folder, filename)
        with rasterio.open(path) as src:
            ndvi = src.read(1)
            ndvi = np.where(ndvi == src.nodata, np.nan, ndvi)  # Mask no-data values

        # # Display NDVI
        # plt.figure(figsize=(10, 8))
        # img = plt.imshow(ndvi, cmap='Spectral', vmin=-0.25, vmax=0.8)  # Use a vegetation-appropriate colormap
        # plt.colorbar(img, label='NDWI')
        # plt.title(f"{filename}")
        # plt.axis('off')
        # plt.show()